this notebook is the base of the code for the artwork [EIN STÜCK „LYRIK“: AUTOPOESIE APPARATUS FÜR ANTIKAPITALISTISCHE WERBEMITTEL](https://exmediawiki.khm.de/exmediawiki/images/3/35/KI-Plakat.pdf) from our former student [Verena Lercher](http://www.verenalercher.com/)  

---

# Text generation mit LSTM


## Implementierung der LSTM-Texterzeugung für Zeichen
![](./data/character-level_neural_language_model.png)

In [9]:
import keras
keras.__version__

'2.6.0'

## Daten vorbereiten

laden und konvertieren in 'lowercase':

In [10]:
import keras
import numpy as np

#eigenes textfile (in UTF-8)
text = open("./data/KeinerWeissMehr.txt").read().lower()
print('Länge des Textkorpus:', len(text))


Länge des Textkorpus: 462081


umschreiben...:
1. sequenzen mit der länge `maxlen` extrahieren
2. mit `one-hot-encoding` in numpy-array `x` umwandeln
3. numpy-array mit der `Shape(sequences, maxlen, unique_characters)` speichern
4. numpy-array `y` für zielwerte vorbereiten

In [11]:
# Exraktion von 60-Zeichen-Sequenzen
maxlen = 60

# sampling einer neuen sequenz nach jeweils 3 zeichen
step = 3

# speichern der extrahierten sequenzen
sentences = []

# speichern der zielwerte (der vorhergesagten zeichen)
next_chars = []

for i in range(0, len(text) - maxlen, step):
    sentences.append(text[i: i + maxlen])
    next_chars.append(text[i + maxlen])
print('Anzahl der Sequenzen:', len(sentences))

# Liste der unterschiedlichen Zeichen erstellen
chars = sorted(list(set(text)))
print('Anzahl der in diesem Text verwendeten Zeichen:', len(chars))
# Dictionary mapping unique characters to their index in `chars`
char_indices = dict((char, chars.index(char)) for char in chars)

# one-hot Kodierung in binäre Arrays
print('Vektorisierung...')
x = np.zeros((len(sentences), maxlen, len(chars)), dtype=np.bool)
y = np.zeros((len(sentences), len(chars)), dtype=np.bool)
for i, sentence in enumerate(sentences):
    for t, char in enumerate(sentence):
        x[i, t, char_indices[char]] = 1
    y[i, char_indices[next_chars[i]]] = 1

Anzahl der Sequenzen: 154007
Anzahl der in diesem Text verwendeten Zeichen: 53
Vektorisierung...


## KNN erzeugen

1. ein einzelner `LSTM` layer 
2. ein `Dense` Klassifizierer
3. Berechnung der softmax-Funktion aller möglichen Zeichen

In [12]:
from keras import layers

model = keras.models.Sequential()
model.add(layers.LSTM(128, input_shape=(maxlen, len(chars))))
model.add(layers.Dense(len(chars), activation='softmax'))

Da die Zielwerte one-hot encoded wurden, 
benutzen wir `categorical_crossentropy` als Verlustfunktion um das Netz zu trainieren:

In [13]:
from tensorflow.keras import optimizers
optimizer = optimizers.RMSprop(lr=0.01)
model.compile(loss='categorical_crossentropy', optimizer=optimizer)

## Trainieren des Sprachmodells und Sampling

Mit trainierten Modell und einem kleinen Anfangstext neue Texte erzeugen...:

1. Wahrscheinlichkeitsverteilung (anhand bereits erzeugten Textes) für das nächste Zeichen erzeugen
2. Neugewichtung für eine bestimmte Temperatur durchführen
3. das nächste Zeichen erzeugen
4. das neue Zeichen am Ende des Textes hizufügen

Code zur Neugewichtung und zum Abrufen eines Zeichenindex (Sampling-Funktion):

In [14]:
def sample(preds, temperature=1.0):
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

Finally - Das Modell trainieren und Text erzeugen

nach jeder Epoche verschieden Temperaturen verwenden

*...zum beobachten wie sich der erzeugte Text entwickelt und wie sich die verschiedenen Temperaturen der Sampling-Stratgie auswirken:*

In [15]:
import random
import sys

# Das Modell 60 Epochen lang trainieren
for epoch in range(1, 60):
    print('epoch', epoch)
    # Anpassung des Modells für einen Trainingsdurchlauf
    model.fit(x, y,
              batch_size=128,
              epochs=1)

    # Zufällige Auswahl eines Anfangstexts
    start_index = random.randint(0, len(text) - maxlen - 1)
    generated_text = text[start_index: start_index + maxlen]
    print('--- Erzeuge Text mit dem Anfangswert: "' + generated_text + '"')
    
    # ausprobieren verschiedener Temperaturen
    for temperature in [0.2, 0.5, 1.0, 1.2]:
        print('------ Temperatur:', temperature)
        sys.stdout.write(generated_text)

        # vom Anfangstext ausgehend 400 Zeichen erzeugen
        for i in range(400):
            sampled = np.zeros((1, maxlen, len(chars)))
            for t, char in enumerate(generated_text):
                # one-hot-codierung der schon erzeugten zeichen
                sampled[0, t, char_indices[char]] = 1.

            preds = model.predict(sampled, verbose=0)[0]
            next_index = sample(preds, temperature)
            # sampling des nächsten Zeichens
            next_char = chars[next_index]

            generated_text += next_char
            generated_text = generated_text[1:]

            sys.stdout.write(next_char)
            sys.stdout.flush()
        print()

epoch 1
1204/1204 [==============================] - 140s 115ms/step - loss: 1.8710
--- Erzeuge Text mit dem Anfangswert: "int? man kann da nichts mehr machen, glaub mir, ein winziger"
------ Temperatur: 0.2
int? man kann da nichts mehr machen, glaub mir, ein winziger das war es war auf den er sich auf der stehen, das war es war er sich sich das war das war auf die schlaffen, das zu sehen, das war er so war er sich so war es war es war das war sie sich auf der schließlich sich das war war er sich so war es war das soch das war er so das war das war er sich das gehonten war es war das war es war es war er sich das war es war es war er sich das zu sehen war das 
------ Temperatur: 0.5
 er sich das war es war es war er sich das zu sehen war das war er wal es nach sie sich das zu den hautend sich auch bis ihm so das das war es war das stell, was das war das und das war aus diesen war sie war, was als das vorber hiere sich vorberen wollte, das war so das war sagen so war das auf der schwiel

KeyboardInterrupt: 